In [1]:
import xgboost as xgb
import lightgbm as lgb
import sklearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels
import statsmodels.api as sm
import sys
import pandas as pd
import numpy as np

In [2]:
print('Python version: {}\n'.format(sys.version))
print("xgboost version: {}".format(xgb.__version__))
print("lightgbm version: {}".format(lgb.__version__))
print('sci-kit learn version: {}'.format(sklearn.__version__))
print('statsmodels version: {}'.format(statsmodels.__version__))
print('matplotlib version: {}'.format(matplotlib.__version__))
print('pandas version: {}'.format(pd.__version__))
print('numpy version: {}'.format(np.__version__))

Python version: 3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 09:55:02) 
[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)]

xgboost version: 0.72.1
lightgbm version: 2.1.2
sci-kit learn version: 0.19.1
statsmodels version: 0.9.0
matplotlib version: 2.2.2
pandas version: 0.23.4
numpy version: 1.14.3


In [3]:
df = pd.read_csv('abalone19.dat', header=None)

In [13]:
df.head(10)

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,negative
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,negative
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,negative
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,negative
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,negative
5,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,negative
6,F,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,negative
7,F,0.545,0.425,0.125,0.7680,0.2940,0.1495,0.260,negative
8,M,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.165,negative
9,F,0.550,0.440,0.150,0.8945,0.3145,0.1510,0.320,positive


In [5]:
df.sample(5)

,0,1,2,3,4,5,6,7,8
2384,F,0.485,0.365,0.140,0.6195,0.2595,0.1445,0.1770,negative
3424,M,0.710,0.550,0.200,1.9045,0.8820,0.4400,0.5000,negative
928,I,0.440,0.325,0.090,0.3500,0.1480,0.0670,0.1050,negative
4171,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,negative
2879,I,0.495,0.375,0.120,0.6140,0.2855,0.1365,0.1610,negative


In [6]:
df.tail()

,0,1,2,3,4,5,6,7,8
4169,M,0.560,0.430,0.155,0.8675,0.4000,0.1720,0.2290,negative
4170,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,negative
4171,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,negative
4172,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,negative
4173,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,negative


In [7]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [8]:
df.dtypes

0     object
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
8     object
dtype: object

In [16]:
df.iloc[9,8]

' positive'

In [18]:
df[8] = df[8].str.strip()

In [19]:
df['target'] = np.select([df[8]=='negative',df[8]=='positive'],[0,1])

In [20]:
df.sample(5)

,0,1,2,3,4,5,6,7,8,target
2934,M,0.625,0.495,0.175,1.2075,0.5310,0.2810,0.3525,negative,0
1268,I,0.470,0.355,0.180,0.4800,0.2055,0.1050,0.1505,negative,0
607,I,0.485,0.370,0.130,0.4580,0.1810,0.1130,0.1360,negative,0
3133,M,0.460,0.350,0.125,0.5165,0.1885,0.1145,0.1850,negative,0
2043,I,0.405,0.310,0.090,0.3120,0.1380,0.0600,0.0870,negative,0


In [21]:
print(len(df['target']))
print(sum(df['target']))

4174
32


In [22]:
df.iloc[9,8]

'positive'

In [25]:
df_dummy = pd.get_dummies(df, columns=[0], drop_first=True)

In [27]:
df_dummy.sample(10)

,1,2,3,4,5,6,7,8,target,0_I,0_M
1309,0.550,0.430,0.145,0.7895,0.3745,0.1710,0.2230,negative,0,1,0
2643,0.490,0.380,0.110,0.5540,0.2935,0.1005,0.1500,negative,0,0,1
3364,0.280,0.210,0.065,0.0905,0.0350,0.0200,0.0300,negative,0,0,1
4014,0.640,0.525,0.175,1.3820,0.6460,0.3115,0.3700,negative,0,0,0
3432,0.370,0.280,0.090,0.2565,0.1255,0.0645,0.0645,negative,0,1,0
431,0.600,0.470,0.155,1.0360,0.4375,0.1960,0.3250,negative,0,0,1
3267,0.595,0.465,0.145,1.1070,0.4020,0.2415,0.3100,negative,0,0,0
1281,0.500,0.375,0.145,0.5795,0.2390,0.1375,0.1850,negative,0,1,0
1703,0.640,0.525,0.185,1.7070,0.7630,0.4205,0.4435,negative,0,0,1
2482,0.460,0.360,0.135,0.6105,0.1955,0.1070,0.2350,negative,0,0,1


In [36]:
X = df_dummy.loc[:,[1,2,3,4,5,6,7,'0_I','0_M']].values

In [39]:
X

array([[0.455 , 0.365 , 0.095 , ..., 0.15  , 0.    , 1.    ],
       [0.35  , 0.265 , 0.09  , ..., 0.07  , 0.    , 1.    ],
       [0.53  , 0.42  , 0.135 , ..., 0.21  , 0.    , 0.    ],
       ...,
       [0.59  , 0.44  , 0.135 , ..., 0.2605, 0.    , 1.    ],
       [0.6   , 0.475 , 0.205 , ..., 0.308 , 0.    , 1.    ],
       [0.625 , 0.485 , 0.15  , ..., 0.296 , 0.    , 0.    ]])

In [40]:
y = df_dummy['target'].values

In [41]:
y

array([0, 0, 0, ..., 0, 0, 0])